## Data Colletion from Twitter

In [4]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import numpy as np
import re
import csv
from collections import Counter
import nltk
from tqdm import tqdm

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [5]:
from datetime import datetime, timedelta

def date_range(start, end):
    start_date = datetime.strptime(start, '%Y-%m-%d').date()
    end_date = datetime.strptime(end, '%Y-%m-%d').date()
    delta = end_date - start_date 
    days = [start_date + timedelta(days=i) for i in range(delta.days + 1)]
    return list(map(lambda n: n.strftime("%Y-%m-%d"), days))

data_collection_period = date_range('2022-08-01', '2022-09-01')

In [6]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\-…]',' ', temp)
    temp = re.sub('&amp;','and', temp)
    temp = re.sub("\n"," ", temp)
    temp = re.sub("\t"," ", temp)
    temp = re.sub("[^a-z0-9À-ž ]","", temp)
    temp = temp.split()
    temp = " ".join(word for word in temp)
    return temp
def get_most_recent_tweets(city, amount, distance = '20km'):
    df_city = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance}').get_items(), amount))[['date', 'content']]
    df_city = df_city.rename(columns={"content": "rawContent"})
    # df_city['cleaned_content'] = [clean_tweet(i) for i in df_city['rawContent']]
    df_city.to_csv(f"data/{city}/{amount}_most_recent_tweets.csv")
    return df_city


def get_tweets_over_period(city, amount_per_day, data_collection_period, distance='20km'):
    df_city = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance} since:{data_collection_period[0]} until:{data_collection_period[1]} ').get_items(), amount_per_day))[['date', 'content']]
    for i in tqdm(range(1, len(data_collection_period)-1)):
        df_temp = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(f'near:"{city}" within:{distance} since:{data_collection_period[i]} until:{data_collection_period[i+1]} ').get_items(), amount_per_day))[['date', 'content']]
        df_city = df_city.append(df_temp)
    # df_city = df_city.rename(columns={"content": "rawContent"})
    df_city.to_csv(f"data/{city}/{amount_per_day*len(data_collection_period)}_tweets_over_period.csv")
    return df_city

def clean_tweets_from_file(file_name):
    df_city = pd.read_csv(file_name, index_col=0)
    df_city['cleaned_content'] = [clean_tweet(i) for i in df_city['rawContent']]
    df_city.to_csv(file_name)
    return df_city


In [7]:
temp_df = clean_tweets_from_file('data/Singapore/tweets_over_period/24400_tweets_over_period.csv')

In [64]:
temp_df.sample(10)
# clean_tweet("I <3 u")

,date,rawContent,cleaned_content
1878,2022-10-04 12:42:24+00:00,faith,faith
696,2022-10-04 12:44:54+00:00,"NEW #Raiders @SNBToday EP. via @AudacySports \n\n-Other than feature Josh Jacobs, what else did the Raiders do to get the W?\n\n-@LVGully and I share our thoughts on the O-line combination \n\n-Thoughts on Darren Waller, Chandler Jones and Amik Robertson\n\n🎧⬇️\nhttps://t.co/gcTghMjY6X",new ep via other than feature josh jacobs what else did the raiders do to get the w and i share our thoughts on the oline combination thoughts on darren waller chandler jones and amik robertson
1307,2022-10-04 12:43:37+00:00,@Go_Blue1985 @SparklesNSunsh2 @Jillie_Alexis @TimRunsHisMouth I keep hearing this from MAGAs but cannot find a single article in Conservative media pointing to the complete &amp; absolute end to all global conflicts between 2016-2020. Can you point me to your sources?,i keep hearing this from magas but cannot find a single article in conservative media pointing to the complete and absolute end to all global conflicts between 20162020 can you point me to your sources
367,2022-10-04 12:45:30+00:00,@tosa05909757 I said slow the pace - not stop or pause. Monetary policy works with a lag of 6 to 18 months. We're in month 7 and the stress caused by strong dollar is serious. Slow to 50 bp plus QT is still a lot of tightening.,i said slow the pace not stop or pause monetary policy works with a lag of 6 to 18 months were in month 7 and the stress caused by strong dollar is serious slow to 50 bp plus qt is still a lot of tightening
1716,2022-10-04 12:42:46+00:00,@pulte I need a lambo to get my wife back #GME #MOON #SHFRFUCKED https://t.co/vgjEq0MYVx,i need a lambo to get my wife back
226,2022-10-04 12:45:50+00:00,"@SignalsToNoises Well, I did write that in the piece, so I agree :) but as I also write, you have to consider what it takes to get a movie greenlit. They don’t just appear out of thin air.",well i did write that in the piece so i agree but as i also write you have to consider what it takes to get a movie greenlit they dont just appear out of thin air
1859,2022-10-04 12:42:27+00:00,How can we help Iran?,how can we help iran
2705,2022-10-04 12:40:42+00:00,I just paid for the weather app… I’m officially an adult.,i just paid for the weather app im officially an adult
2412,2022-10-04 12:41:17+00:00,"@TomCurle @nathaliejacoby1 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 You’re hilarious! Justice and the freedom to be is such divisive, hateful behavior!",youre hilarious justice and the freedom to be is such divisive hateful behavior
1203,2022-10-04 12:43:51+00:00,@sadie65 @jujutough @YasinShanks Fr. Bro a page one Stan 🤣,fr bro a page one stan


In [33]:
%%time
# get_most_recent_tweets("Singapore", 3000, distance='20km')
get_tweets_over_period("New York", 1000, data_collection_period)

KeyboardInterrupt: 

In [26]:
%%time
get_most_recent_tweets("New York", 3000, distance='20km')

CPU times: user 1.75 s, sys: 85 ms, total: 1.83 s
Wall time: 2min 4s


,date,rawContent,cleaned_content
0,2022-10-04 12:46:15+00:00,@ManotoNews خواهراش میگه که میخوان مث مادرش باشن\n#مهسا_امینی \n#اعتصابات_سراسری \n#OpIran,
1,2022-10-04 12:46:14+00:00,🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡🤡 https://t.co/U8HadmmAO8,
2,2022-10-04 12:46:14+00:00,Simply Be Well -Six Plant Based Body Soaps - Boxed Set NEW https://t.co/AmoTUCY5vY #eBay via @eBay,simply be well six plant based body soaps boxed set new via
3,2022-10-04 12:46:14+00:00,"$IDEX - Ideanomics, Solectrac secured new business contracts to supply electric tractors to major fleet operators https://t.co/3sOiHMyi9S",idex ideanomics solectrac secured new business contracts to supply electric tractors to major fleet operators
4,2022-10-04 12:46:14+00:00,@DrewOnline @reggaeology What petcentage of motorbikes on the road are legally licensed?,what petcentage of motorbikes on the road are legally licensed
...,...,...,...
2995,2022-10-04 12:40:08+00:00,@jeongsteph You’ve been manifesting this haha congrats @jeongsteph !!!,youve been manifesting this haha congrats
2996,2022-10-04 12:40:08+00:00,"@Gokul_Sahni Therein lies the trap of aligning with the West—your prosperity, dependent on Western technology and technology, is subject to adherence to their vision of the world. That vision, or “rules”, codifies Western dominance. Challenge that and you’re finished.",therein lies the trap of aligning with the westyour prosperity dependent on western technology and technology is subject to adherence to their vision of the world that vision or rules codifies western dominance challenge that and youre finished
2997,2022-10-04 12:40:08+00:00,@appare_noka 外で気を付けてね,
2998,2022-10-04 12:40:07+00:00,Putin’s war comes home to Russia | @MID_RF ☠️ #KILLPutinNOW ⚰️ &amp; all his #Oligarchs 🤑 + Kremlin Worms 🪱 &amp; DEAD Men Walking Yevgeny Prigozhin #YevgenyPrigozhin &amp; Ramzan Kadyrov #RamzanKadyrov https://t.co/TKtN39OJMU,putins war comes home to russia amp all his kremlin worms amp dead men walking yevgeny prigozhin amp ramzan kadyrov
